In [1]:
import pickle
from dataset import CustomDataset
from model import MyModel
import torch
from tqdm.notebook import tqdm

%load_ext autoreload
%autoreload 2

In [6]:
with open('../pos', 'rb') as file:
    t = pickle.load(file)

In [3]:
dataset = ds.CustomDataset('../train.data', t=t, java_path="C:\\Program Files\\Common Files\\Oracle\\Java\\javapath")

loading data...
removing length outliers...
100%|██████████| 68645/68645 [00:01<00:00, 67992.86it/s]


In [4]:
model = model.MyModel(len(dataset.tokens) + 1, 32, 128, [256, 128, 64])

In [15]:
x = dataset[1:110]
o = torch.sigmoid(model(x[0], torch.LongTensor(x[1])).squeeze())

In [16]:
o.shape

torch.Size([109, 91])

In [17]:
from functions import binary_acc

In [18]:
binary_acc(o, torch.Tensor(x[2]))

torch.Size([109, 91]) torch.Size([109, 91])


tensor(81.)

In [1]:
from functions import generate_model

Running on the GPU


In [22]:
e = generate_model('../train.data', t=t, java_path="C:\\Program Files\\Common Files\\Oracle\\Java\\javapath")

Running on the GPU
proccessing data...
loading data...
  0%|          | 0/68645 [00:00<?, ?it/s]removing length outliers...
embedding data...
100%|██████████| 68645/68645 [00:00<00:00, 71203.79it/s]


epoch 1 | loss: 2.15824 | accuracy: 94.55%


epoch 2 | loss: 2.12908 | accuracy: 94.57%


epoch 3 | loss: 2.12572 | accuracy: 94.61%


epoch 4 | loss: 2.12430 | accuracy: 94.63%


...

epoch 6 | loss: 2.12214 | accuracy: 94.66%


epoch 7 | loss: 2.12157 | accuracy: 94.67%


epoch 8 | loss: 2.12073 | accuracy: 94.70%


epoch 9 | loss: 2.12051 | accuracy: 94.71%


epoch 10 | loss: 2.11987 | accuracy: 94.73%

Finished Training


In [23]:
tokens, t, model = e

In [8]:
with open('pos', 'wb') as file:
    pickle.dump(t, file)

In [25]:
torch.save(model.state_dict(), 'model.pt')

In [3]:
with open('tokens.pickle', 'rb') as file:
    tokens = pickle.load(file)

In [4]:
tokens

{1: 'oov',
 'N': 2,
 'DELM': 3,
 'ADJ': 4,
 'PO': 5,
 'CON': 6,
 'AR': 7,
 'ADV': 8,
 'V_PR': 9,
 '.': 10,
 'DET': 11,
 'V_PA': 12,
 'PRO': 13,
 'PS': 14,
 'IF': 15,
 'MORP': 16,
 'PP': 17,
 'FW': 18,
 'OH': 19,
 'INT': 20,
 'NN': 21}

In [8]:
with open('./pos', 'rb') as file:
    t = pickle.load(file)

In [9]:
dataset = CustomDataset('../test.data', tokens=tokens, t=t, java_path="C:\\Program Files\\Common Files\\Oracle\\Java\\javapath")

loading data...
100%|██████████| 6431/6431 [00:00<00:00, 79371.38it/s]removing length outliers...
embedding data...



In [5]:
java_path="C:\\Program Files\\Common Files\\Oracle\\Java\\javapath"

In [6]:
from parsivar import POSTagger
import ray
ray.init(ignore_reinit_error=True)

2021-06-04 14:20:17,356	INFO services.py:1267 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '127.0.0.1',
 'raylet_ip_address': '127.0.0.1',
 'redis_address': '127.0.0.1:6379',
 'object_store_address': 'tcp://127.0.0.1:51201',
 'raylet_socket_name': 'tcp://127.0.0.1:9664',
 'webui_url': '127.0.0.1:8265',
 'session_dir': 'C:\\Users\\moham\\AppData\\Local\\Temp\\ray\\session_2021-06-04_14-20-11_849712_32500',
 'metrics_export_port': 25985,
 'node_id': '122c37499235af69ed3647d5edc3f1b64938bb12ecd4daec9cb9a601'}

In [7]:
tagger = POSTagger(tagging_model="wapiti", jdk_variable_path=java_path)
@ray.remote
def transform(x):
    i, j = x
    a = tagger.parse(j)
    with open('./t/' + str(i) + '.txt', 'wb') as file:
        pickle.dump(a, file)
    return a
futures = [transform.remote(i) for i in enumerate(dataset.x)]
t = ray.get(futures)

In [ ]:
t

In [10]:
model = MyModel(len(tokens) + 1, 32, 128, [256, 128, 64])
model.load_state_dict(torch.load('model.pt'))

<All keys matched successfully>

In [12]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")


Running on the GPU


In [25]:
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum / (y_test.shape[0])
    acc = torch.round(acc * 100)
    
    return acc

In [14]:
test_loader = torch.utils.data.DataLoader(dataset, batch_size=64)

In [16]:
model = model.to(device)

In [32]:
real = []
pred = []

with torch.no_grad():

    model.eval()

    for lengths, inputs, labels in tqdm(test_loader, desc='Batches', leave=False):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(lengths, inputs).squeeze()
        for i, l in enumerate(lengths):
            pred += torch.round(torch.sigmoid(outputs[i])).detach().cpu().tolist()
            real += labels[i].detach().cpu().tolist()

In [30]:
from sklearn.metrics import classification_report

In [33]:
print(classification_report(real, pred))

              precision    recall  f1-score   support

         0.0       0.93      1.00      0.96    560103
         1.0       0.97      0.01      0.02     44411

    accuracy                           0.93    604514
   macro avg       0.95      0.50      0.49    604514
weighted avg       0.93      0.93      0.89    604514



In [12]:
from functions import *

In [36]:
dataset

In [13]:
compute_accuracy(dataset, model)

tensor(92.7426, device='cuda:0')